In [ ]:
from pathlib import Path


import matplotlib.pyplot as plt

In [ ]:
from toolbox_continu_inzicht import Config, DataAdapter

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "integrate_statistics_comb.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

Bereken een GEKB curve met data uit de database

In [ ]:
from toolbox_continu_inzicht.fragility_curves import FragilityCurveOvertoppingMultiple

In [ ]:
fragility_curves_overtopping = FragilityCurveOvertoppingMultiple(
    data_adapter=data_adapter
)
fragility_curves_overtopping.run(
    input=["slopes", "profiles", "bedlevel"],
    output="fragility_curves",
)

In [ ]:
fragility_curves_overtopping.df_out

##### deze wordt toegevoegd aan de database, die hele tabel kunnen we ook weer ophalen uit de database 

In [ ]:
df_all_fragility_curves = data_adapter.input("fragility_curves")

In [ ]:
df_all_fragility_curves

In [ ]:
lst_df_717_fragility_curves = []
failure_mechanism_types = ["HTKW", "PKW", "STKWl", "GEKB"]
for failure_mechanism_type in failure_mechanism_types:
    lst_df_717_fragility_curves.append(
        df_all_fragility_curves.query(
            "`section_id` == 717 & `failuremechanism` == @failure_mechanism_type"
        )
    )

In [ ]:
fig, ax = plt.subplots()
for fc in lst_df_717_fragility_curves:
    ax.plot(
        fc["hydraulicload"],
        fc["failureprobability"],
        label=fc["failuremechanism"].unique()[0],
        alpha=0.6,
    )
ax.legend()
ax.set_xlim((21, 23))

In [ ]:
from toolbox_continu_inzicht.fragility_curves import CombineFragilityCurvesDependent

In [ ]:
for name, df in zip(failure_mechanism_types, lst_df_717_fragility_curves):
    df = df.rename(columns={"failureprobability": "failure_probability"}).copy()
    data_adapter.set_dataframe_adapter(name, df, if_not_exist="create")

In [ ]:
combine_fragility_curves = CombineFragilityCurvesDependent(data_adapter)

In [ ]:
lst_df_717_fragility_curves[0]

In [ ]:
combine_fragility_curves.run(failure_mechanism_types, "fragility_curves_combined_csv")
output_combined_curve = combine_fragility_curves.df_out

In [ ]:
fig, ax = plt.subplots()
for fc in lst_df_717_fragility_curves:
    ax.plot(
        fc["hydraulicload"],
        fc["failure_probability"],
        label=fc["failuremechanism"].unique()[0],
        alpha=0.6,
    )
# ax.set_xlim((21, 23))
ax.plot(
    output_combined_curve["hydraulicload"],
    output_combined_curve["failure_probability"],
    label="combined",
    color="black",
)
ax.legend()

In [ ]:
from toolbox_continu_inzicht.fragility_curves import IntegrateFragilityCurve

In [ ]:
integrate_statistics_per_section = IntegrateFragilityCurve(data_adapter=data_adapter)
integrate_statistics_per_section.run(
    input=["exceedance_curve_csv", "fragility_curves_combined_csv"], output="result"
)

In [ ]:
ax2 = ax.twinx()
data_adapter.input("exceedance_curve_csv").plot(x="hydraulicload", ax=ax2, logy=True)
ax2.legend().remove()
handles, labels = ax.get_legend_handles_labels()
handles_, labels_ = ax2.get_legend_handles_labels()
ax.legend(handles + handles_, labels + labels_)
fig

In [ ]:
initial_integrated_curve = integrate_statistics_per_section.df_out
fig, ax = plt.subplots()
ax.bar(
    initial_integrated_curve["hydraulicload"],
    initial_integrated_curve["probability_contribution"],
    width=0.1,
)
xlim = [21, 25]
ax.set_xlim(xlim)

In [ ]:
from toolbox_continu_inzicht.base.fragility_curve import FragilityCurve

In [ ]:
fragility_curve = FragilityCurve(data_adapter=data_adapter)

In [ ]:
fragility_curve.load("fragility_curves_combined_csv")

In [ ]:
first_fragility_curve = fragility_curve.as_dataframe()

In [ ]:
update_level = 22
trust_factor = 1
fragility_curve.reliability_update(update_level=update_level, trust_factor=trust_factor)

In [ ]:
updated_curve = fragility_curve.as_dataframe()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(
    first_fragility_curve["hydraulicload"],
    first_fragility_curve["failure_probability"],
    label="Originele gecombineerde curve",
)
ax.plot(
    updated_curve["hydraulicload"],
    updated_curve["failure_probability"],
    ls="--",
    label="Aangepaste combined curve",
)
ax.legend()
ax.set_xlim(xlim)

In [ ]:
data_adapter.set_dataframe_adapter(
    "updated_fragility_curve", fragility_curve.as_dataframe(), if_not_exist="create"
)

In [ ]:
integrate_statistics_per_section = IntegrateFragilityCurve(data_adapter=data_adapter)
integrate_statistics_per_section.run(
    input=["exceedance_curve_csv", "updated_fragility_curve"], output="result"
)

In [ ]:
df_out = integrate_statistics_per_section.df_out
fig, ax = plt.subplots()
ax.bar(
    initial_integrated_curve["hydraulicload"],
    initial_integrated_curve["probability_contribution"],
    width=0.1,
)
ax.bar(df_out["hydraulicload"], df_out["probability_contribution"], width=0.1)
ax.set_xlim(xlim)
ax.set_xlabel("Water level")
ax.set_ylabel("Probability contribution");

 #### beheerdersoordeel

In [ ]:
from ipywidgets import interact

xlim = [20, 25]


@interact(update_level=(xlim[0], xlim[-1], 0.1), trust_factor=(0, 1, 0.1))
def make_plot(update_level, trust_factor):
    integrate_statistics_per_section = IntegrateFragilityCurve(
        data_adapter=data_adapter
    )
    integrate_statistics_per_section.run(
        input=["exceedance_curve_csv", "fragility_curves_combined_csv"], output="result"
    )
    fragility_curve = FragilityCurve(data_adapter=data_adapter)
    fragility_curve.load("fragility_curves_combined_csv")
    fragility_curve.reliability_update(
        update_level=update_level, trust_factor=trust_factor
    )
    data_adapter.set_dataframe_adapter(
        "updated_fragility_curve", fragility_curve.as_dataframe(), if_not_exist="create"
    )
    integrate_statistics_per_section = IntegrateFragilityCurve(
        data_adapter=data_adapter
    )
    integrate_statistics_per_section.run(
        input=["exceedance_curve_csv", "updated_fragility_curve"], output="result"
    )

    df_out = integrate_statistics_per_section.df_out
    fig, ax = plt.subplots()
    ax.bar(
        initial_integrated_curve["hydraulicload"],
        initial_integrated_curve["probability_contribution"],
        width=0.1,
    )
    ax.bar(df_out["hydraulicload"], df_out["probability_contribution"], width=0.1)
    ax.set_xlim(xlim)
    ax.set_xlabel("Water level")
    ax.set_ylabel("Probability contribution")
    ax.set_title(
        "Updated probability contribution with given update level and trust factor"
    )